# HRA – Aufschreibung bereinigen

Bereinigung & Vereinheitlichung (Spalten, Datum/Zeit, Station/OP-Splitting, Freitext).

Zusätzlich werden Mapping-Tabellen für die Freitext-Vereinheitlichung in `../data/processed/` gespeichert.

In [1]:
import re
import unicodedata
import datetime
import numpy as np
import pandas as pd
from rapidfuzz import process, fuzz

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200)

# -------------------------
# 1) Spalten / Text Helpers
# -------------------------
def _normalize_colname(c: object) -> str:
    c = "" if c is None else str(c)
    c = unicodedata.normalize("NFKC", c)
    c = c.replace("\n", " ")
    c = re.sub(r"\s+", " ", c).strip()
    c = re.sub(r"[‐-‒–—―]", "-", c)           # Bindestrich-Varianten vereinheitlichen
    c = re.sub(r"\s*/\s*", "/ ", c)          # einheitliche Slash-Schreibweise
    c = re.sub(r"\s+", " ", c).strip()
    return c

_CANON_PATTERNS = [
    (r"^station\s*/\s*op$", "Station/ OP"),
    (r"^station/op$", "Station/ OP"),
    (r"^datum\s*neu$", "DatumNEU"),
    (r"^zeit\s*von$", "Zeit von"),
    (r"^zeit\s*bis$", "Zeit bis"),
    (r"^unterbrechungsursache$", "Unterbrechungsursache"),
    (r"^bemerkung$", "Bemerkung"),
    (r"^dauer\s*org-?\s*mangel$", "Dauer Org-Mangel"),
]

def canonicalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    cols = [_normalize_colname(c) for c in df.columns]
    canon = []
    for c in cols:
        c2 = c
        for pat, repl in _CANON_PATTERNS:
            if re.match(pat, c2, flags=re.IGNORECASE):
                c2 = repl
                break
        canon.append(c2)

    # Doppelte Spaltennamen abfangen
    seen = {}
    out = []
    for c in canon:
        if c not in seen:
            seen[c] = 0
            out.append(c)
        else:
            seen[c] += 1
            out.append(f"{c}_{seen[c]}")
    df = df.copy()
    df.columns = out
    return df

def find_col(cols, patterns) -> str | None:
    for pat in patterns:
        for c in cols:
            if re.match(pat, c, flags=re.IGNORECASE):
                return c
    # fallback (sehr tolerant)
    for c in cols:
        lc = c.lower()
        if "station" in lc and "op" in lc:
            return c
    return None

def normalize_free_text(s: pd.Series) -> pd.Series:
    s = s.astype("string")
    s = s.map(lambda x: unicodedata.normalize("NFKC", x) if pd.notna(x) else x)
    s = s.str.lower()
    s = s.str.replace(r"\s+", " ", regex=True).str.strip()
    s = s.str.replace(r"[‐-‒–—―]", "-", regex=True)
    s = s.str.replace(r"[•·●]", " ", regex=True)
    s = s.str.replace(r"\s+", " ", regex=True).str.strip()
    s = s.replace({"": pd.NA, "nan": pd.NA, "none": pd.NA, "k.a.": pd.NA, "k. a.": pd.NA})
    return s

def fuzzy_standardize(norm_s: pd.Series, threshold: int = 97, min_count: int = 2):
    # Vereinheitlicht NUR sehr ähnliche Schreibweisen (Typo/Spacing).
    counts = norm_s.dropna().value_counts()
    variants = counts.index.tolist()

    mapping = {}
    for v in variants:
        if v in mapping:
            continue
        mapping[v] = v  # rep
        matches = process.extract(v, variants, scorer=fuzz.token_sort_ratio, score_cutoff=threshold, limit=None)
        for m, score, _ in matches:
            if m not in mapping:
                mapping[m] = v

    std = norm_s.map(mapping).astype("string")
    std = std.where(norm_s.notna(), pd.NA)

    map_df = pd.DataFrame({
        "original": list(mapping.keys()),
        "standard": list(mapping.values()),
        "count": [counts.get(k, 0) for k in mapping.keys()]
    }).sort_values(["standard", "count"], ascending=[True, False])

    if min_count > 1:
        map_df = map_df[map_df["count"] >= min_count].copy()

    return std, map_df

# -------------------------
# 2) Datum / Zeit Helpers
# -------------------------
def parse_excel_date_to_date(s: pd.Series) -> pd.Series:
    dt = pd.to_datetime(s, errors="coerce")
    return dt.dt.normalize()

def parse_excel_time_to_str(s: pd.Series) -> pd.Series:
    def conv(x):
        if pd.isna(x):
            return pd.NA
        if isinstance(x, datetime.time):
            return x.strftime("%H:%M:%S")
        if isinstance(x, (pd.Timestamp, datetime.datetime)):
            return x.time().strftime("%H:%M:%S")
        if isinstance(x, (float, int, np.floating, np.integer)):
            seconds = int(round(float(x) * 24 * 3600)) % (24 * 3600)
            h = seconds // 3600
            m = (seconds % 3600) // 60
            sec = seconds % 60
            return f"{h:02d}:{m:02d}:{sec:02d}"
        txt = str(x).strip()
        if not txt:
            return pd.NA
        t = pd.to_datetime(txt, errors="coerce")
        if pd.isna(t):
            return pd.NA
        return t.time().strftime("%H:%M:%S")
    return s.map(conv).astype("string")

def time_str_to_minutes(s: pd.Series) -> pd.Series:
    def conv(x):
        if pd.isna(x):
            return np.nan
        parts = str(x).split(":")
        if len(parts) < 2:
            return np.nan
        h = int(parts[0]); m = int(parts[1]); sec = int(parts[2]) if len(parts) > 2 else 0
        return h * 60 + m + sec / 60
    return s.map(conv).astype(float)

# -------------------------
# 3) Station/OP split Helpers
# -------------------------
def _clean_station_token(token: object) -> str:
    t = unicodedata.normalize("NFKC", str(token))
    t = t.strip()
    t = re.sub(r"(?i)\b(R|OP)\.\b", r"\1 ", t)     # Punkt nach R/OP -> Space
    t = re.sub(r"(?i)\b(R|OP)\.", r"\1 ", t)
    t = re.sub(r"(?i)\b(R|OP)\s*([0-9])", r"\1 \2", t)  # R12 -> R 12
    t = re.sub(r"\s+", " ", t).strip()
    t = re.sub(r"(?i)^\s*op\b", "OP", t)
    t = re.sub(r"(?i)^\s*r\b", "R", t)
    return t

def split_station_op_simple(x: object) -> list[str]:
    if pd.isna(x):
        return []
    t = _clean_station_token(x)
    t = re.sub(r"[,/]", "|", t)
    parts = [_clean_station_token(p) for p in t.split("|")]
    return [p for p in parts if p and p.lower() not in ("nan", "none")]

def split_station_op_mta(x: object) -> list[str]:
    # MTA-Spezial:
    # - Trennung bei , oder /
    # - Zusätzlich: wenn in einem Chunk ein 2. 'R' oder 'OP' auftaucht, beginnt ein neuer Wert.
    if pd.isna(x):
        return []
    t = _clean_station_token(x)
    t = re.sub(r"[,/]", "|", t)
    chunks = [c.strip() for c in t.split("|") if c.strip()]

    out = []
    for ch in chunks:
        matches = list(re.finditer(r"(?i)\b(?:R|OP)\b", ch))
        if len(matches) <= 1:
            out.append(_clean_station_token(ch))
        else:
            pos = [m.start() for m in matches]
            for i, p in enumerate(pos):
                end = pos[i + 1] if i + 1 < len(pos) else len(ch)
                seg = ch[p:end].strip()
                if seg:
                    out.append(_clean_station_token(seg))

    seen = set()
    final = []
    for v in out:
        if v and v not in seen:
            seen.add(v)
            final.append(v)
    return final

def expand_split_columns(df: pd.DataFrame, source_col: str, splitter, prefix: str = "Station/ OP") -> pd.DataFrame:
    lists = df[source_col].map(splitter)
    max_len = int(lists.map(len).max()) if len(lists) else 0

    df2 = df.copy()
    df2[f"{prefix}_raw"] = df2[source_col].astype("string")

    for i in range(max_len):
        df2[f"{prefix}_{i+1}"] = lists.map(lambda L: L[i] if len(L) > i else pd.NA).astype("string")

    if max_len > 0:
        df2[source_col] = df2[f"{prefix}_1"]
    else:
        df2[source_col] = df2[source_col].astype("string")

    return df2

def drop_rows_empty_from(df: pd.DataFrame, start_col: str) -> tuple[pd.DataFrame, list[str]]:
    cols = list(df.columns)
    start_idx = cols.index(start_col)
    cols_from = cols[start_idx:]

    df2 = df.copy()
    for c in cols_from:
        if df2[c].dtype == object or str(df2[c].dtype).startswith("string"):
            df2[c] = df2[c].astype("string").str.strip()
            df2.loc[df2[c].isin(["", "nan", "NaN", "None"]), c] = pd.NA

    keep = df2[cols_from].notna().any(axis=1)
    return df2.loc[keep].copy(), cols_from

In [2]:
FILE_PATH = r"../data/raw/Stoerliste_Heckrungenanlage_2023_NEU.xlsx"
SHEET_NAME = "Aufschreibung"

In [3]:
# -------------------------
# HRA: Laden + Bereinigen
# -------------------------
df_raw = pd.read_excel(FILE_PATH, sheet_name=SHEET_NAME)
df = canonicalize_columns(df_raw)

station_col = find_col(df.columns, [r"^Station/\s*OP$"])
if station_col is None:
    raise ValueError("Spalte 'Station/ OP' nicht gefunden. Bitte Spaltennamen prüfen.")

# Zeilen entfernen, die ab Station/OP komplett leer sind
df, cols_from = drop_rows_empty_from(df, station_col)

# Station/OP aufspalten (nur Trennung , /)
df = expand_split_columns(df, source_col=station_col, splitter=split_station_op_simple, prefix="Station/ OP")

# Freitext vereinheitlichen (leicht + sehr konservatives Fuzzy)
for free_col in ["Bemerkung", "Unterbrechungsursache"]:
    if free_col in df.columns:
        df[f"{free_col}_norm"] = normalize_free_text(df[free_col])
        df[f"{free_col}_std"], map_df = fuzzy_standardize(df[f"{free_col}_norm"], threshold=97, min_count=2)
        display(map_df.head(50))
        safe = re.sub(r"[^a-z0-9]+", "_", free_col.lower())
        map_df.to_excel(f"../data/processed/hra_mapping_{safe}.xlsx", index=False)

display(df.head(10))
print("Bereinigt:", df.shape)

,original,standard,count
131,1 ma u - profile schleifen,1 ma u - profile schleifen,3
43,2 ma hr gestelle umpacken / umbuchen,2 ma hr gestelle umpacken / umbuchen,6
93,2 ma hr gestelle umpacken /umbuchen,2 ma hr gestelle umpacken /umbuchen,3
54,2 schweißer,2 schweißer,5
163,2 sensoren defekt rob tech informiert,2 sensoren defekt rob tech informiert,2
183,3. scharnierlager,3. scharnierlager,2
215,3x stromdüse festgeschweißt. 2x draht gewechse...,3x stromdüse festgeschweißt. 2x draht gewechse...,2
172,"ab 14:30 kein bediener, schweißer machen reini...","ab 14:30 kein bediener, schweißer machen reini...",2
227,ablageposition vom deckblech geteacht,ablageposition vom deckblech geteacht,2
23,abschaltsicherung,abschaltsicherung,14


,original,standard,count
26,anlage mit material befüllen,anlage mit material befüllen,7
21,aufträge einseitig,aufträge einseitig,23
10,bauteil fallen gelassen,bauteil fallen gelassen,95
36,bauteil nicht aufgenommen,bauteil nicht aufgenommen,3
43,bauteil nicht gelöst,bauteil nicht gelöst,2
2,bauteil nicht gespannt,bauteil nicht gespannt,715
28,bauteil räumen,bauteil räumen,5
34,bauteilerkennung,bauteilerkennung,3
24,bauteilerkennung gereinigt,bauteilerkennung gereinigt,9
19,brenner gereinigt,brenner gereinigt,24


,Datum,Wochentag,DatumNEU,KW,Schicht,Zeit von,Zeit bis,Dauer Arbeits-zeit,Anzahl MA,Menge N.i. O.,Profi,AHV,Menge Gesamt (Stück),Dauer Org-Mangel,Dauer Anlagen-Ausfall,Störung aufgrund Vormaterial,Dauer Anlagen-Ausfall intern,Dauer Logistik- Defizite,Station/ OP,Unterbrechungsursache,Bemerkung,Fehlercode,Station/ OP_raw,Station/ OP_1,Station/ OP_2,Station/ OP_3,Station/ OP_4,Station/ OP_5,Bemerkung_norm,Bemerkung_std,Unterbrechungsursache_norm,Unterbrechungsursache_std
1,2023-01-05,4.0,2023-01-05 00:00:00,2023/01,t,07:00:00,08:00:00,60.0,2.0,NaN,2.0,NaN,2.0,NaN,NaN,NaN,15.0,NaN,<NA>,Freitext,Gestelle anmelden,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,gestelle anmelden,gestelle anmelden,freitext,freitext
2,2023-01-05,4.0,2023-01-05 00:00:00,2023/01,t,07:00:00,08:00:00,60.0,2.0,NaN,2.0,NaN,2.0,NaN,NaN,NaN,20.0,NaN,<NA>,Wartungsplan,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,wartungsplan,wartungsplan
4,2023-01-05,4.0,2023-01-05 00:00:00,2023/01,t,07:00:00,08:00:00,60.0,2.0,NaN,2.0,NaN,2.0,NaN,5.0,NaN,NaN,NaN,R 03,Freitext,"Programmfehler, Greifer defekt, räumen",NaN,R 03,R 03,<NA>,<NA>,<NA>,<NA>,"programmfehler, greifer defekt, räumen","programmfehler, greifer defekt, räumen",freitext,freitext
5,2023-01-05,4.0,2023-01-05 00:00:00,2023/01,t,08:00:00,09:00:00,60.0,2.0,NaN,2.0,NaN,2.0,NaN,60.0,NaN,NaN,NaN,R 03,Freitext,"Programmfehler, Greifer defekt, räumen",NaN,R 03,R 03,<NA>,<NA>,<NA>,<NA>,"programmfehler, greifer defekt, räumen","programmfehler, greifer defekt, räumen",freitext,freitext
9,2023-01-05,4.0,2023-01-05 00:00:00,2023/01,t,09:15:00,10:00:00,45.0,2.0,NaN,2.0,NaN,2.0,NaN,45.0,NaN,NaN,NaN,R 03,Freitext,"Programmfehler, Greifer defekt, räumen",NaN,R 03,R 03,<NA>,<NA>,<NA>,<NA>,"programmfehler, greifer defekt, räumen","programmfehler, greifer defekt, räumen",freitext,freitext
13,2023-01-05,4.0,2023-01-05 00:00:00,2023/01,t,10:00:00,11:00:00,60.0,2.0,NaN,1.0,NaN,1.0,NaN,15.0,NaN,NaN,NaN,R 03,Freitext,"Programmfehler, Greifer defekt, räumen",NaN,R 03,R 03,<NA>,<NA>,<NA>,<NA>,"programmfehler, greifer defekt, räumen","programmfehler, greifer defekt, räumen",freitext,freitext
14,2023-01-05,4.0,2023-01-05 00:00:00,2023/01,t,10:00:00,11:00:00,60.0,2.0,NaN,1.0,NaN,1.0,NaN,30.0,NaN,NaN,NaN,R 06,Zündfehler,"Drahtseele durchblasen, Draht neu verlegen",NaN,R 06,R 06,<NA>,<NA>,<NA>,<NA>,"drahtseele durchblasen, draht neu verlegen","drahtseele durchblasen, draht neu verlegen",zündfehler,zündfehler
17,2023-01-05,4.0,2023-01-05 00:00:00,2023/01,t,11:00:00,12:00:00,60.0,2.0,1.0,2.0,NaN,3.0,NaN,5.0,NaN,NaN,NaN,R 07,Zündfehler,<NA>,NaN,R 07,R 07,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,zündfehler,zündfehler
18,2023-01-05,4.0,2023-01-05 00:00:00,2023/01,t,11:00:00,12:00:00,60.0,2.0,1.0,2.0,NaN,3.0,NaN,10.0,NaN,NaN,NaN,R 02,Freitext,"Deckblech fallen gelassen, Schanierhalter schief",NaN,R 02,R 02,<NA>,<NA>,<NA>,<NA>,"deckblech fallen gelassen, schanierhalter schief","deckblech fallen gelassen, schanierhalter schief",freitext,freitext
25,2023-01-05,4.0,2023-01-05 00:00:00,2023/01,t,13:30:00,14:00:00,30.0,3.0,NaN,7.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,R 07,Gasdüse getauscht,<NA>,NaN,R 07,R 07,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,gasdüse getauscht,gasdüse getauscht


Bereinigt: (4779, 32)


In [4]:
# -------------------------
# Export
# -------------------------
OUT_CSV = r"../data/processed/aufschreibung_hra_clean.csv"
OUT_XLSX = r"../data/processed/aufschreibung_hra_clean.xlsx"

df.to_csv(OUT_CSV, index=False)
df.to_excel(OUT_XLSX, index=False)
print("Gespeichert:", OUT_CSV, "und", OUT_XLSX)

Gespeichert: ../data/processed/aufschreibung_hra_clean.csv und ../data/processed/aufschreibung_hra_clean.xlsx
